In [0]:
from pyspark.sql.functions import to_date, first, col, round

lendo os arquivos

In [0]:
df_junto = spark.read.parquet("dbfs:/databricks-results/bronze/*/*/*")

In [0]:
moedas = ['USD', 'EUR', 'GBP']

df_moedas = df_junto.filter(df_junto.moeda.isin(moedas))

corrigindo schemas

In [0]:
#corrigindo o schema da data
df_moedas = df_moedas.withColumn("data", to_date("data"))



In [0]:
#Agrupar por data e criar coluna para cada moeda
resultado_taxas_conversao = df_moedas.groupBy("data") \
           .pivot("moeda") \
           .agg(first("taxa")) \
           .orderBy("data", ascending=False)

resultado_taxas_conversao.show()


In [0]:
resultado_valores_reais = resultado_taxas_conversao.select("*")

In [0]:
#transformando o valor da taxaem valor de quantos reais compra a moeda e não o contrario
for moeda in moedas: 
        resultado_valores_reais = resultado_valores_reais.withColumn(
                                                moeda, round(1/col(moeda), 4)
                                                )

In [0]:
# Em uma nova célula colocamos resultado_taxas_conversao = resultado_taxas_conversao.coalesce(1), essa forma juntamos o conjunto de dados em apenas um
resultado_taxas_conversao = resultado_taxas_conversao.coalesce(1)
resultado_valores_reais = resultado_valores_reais.coalesce(1)

In [0]:
#salvando o csv na camada prata 
resultado_taxas_conversao.write\
    .mode ("overwrite")\
    .format("csv")\
    .option("header", "true")\
    .save("dbfs:/databricks-results/prata/taxas_conversao")
    
resultado_valores_reais.write\
    .mode ("overwrite")\
    .format("csv")\
    .option("header", "true")\
    .save("dbfs:/databricks-results/prata/valores_reais")

In [0]:
display(dbutils.fs.ls("dbfs:/databricks-results/prata/valores_reais/"))

In [0]:
# dbutils.fs.rm("dbfs:/databricks-results/prata/taxas_conversao", True)
# dbutils.fs.rm("dbfs:/databricks-results/prata/valores_reais", True)